### Bayesian Optimization with GPs & EI for ResNet50
###### 黄思铭 2020/10/20
##### 步骤：
1. 搭建迁移学习模型与数据管道
2. 定义核心贝叶斯优化函数  
    - Regression Model: Gaussian Processes
    - Acquisition Function: Expected-Improvement(EI)
3. 获取数据集，获取初始采集点
4. 迭代并获取最佳X值

In [128]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
import math
import json

from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from sklearn.gaussian_process import GaussianProcessRegressor
#使用了Matern Kernel作为回归模型的核
from sklearn.gaussian_process.kernels import ConstantKernel, Matern, WhiteKernel
from scipy.optimize import minimize
from scipy.stats import norm

##### 1. 搭建迁移学习与数据管道

In [129]:
'''
初始化全局变量
'''
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNEL = 3
MARK_SIZE = 32 #特征点个数

EPOCHS=1
BATCH_SIZE=32

class BestLogger(keras.callbacks.Callback):
    '''
    回调函数
    '''
    def __init__(self, file):
        super(BestLogger, self).__init__()
        self.file = file
        self.best = float("inf")

    def on_epoch_end(self, batch, logs={}):
        loss = logs.get('val_loss')
        acc = logs.get('val_acc')
        if math.isinf(self.best) or loss < self.best:
            self.best = loss
            with open(self.file, "w") as text_file:
                obj = {
                    'batch' : batch,
                    'val_loss' : loss,
                    'val_acc' : acc
                }
                print(json.dumps(obj), text_file)
                

def get_compiled_model(output_size, learning_rate):
    '''
    定义模型，将learning_rate作为参数
    '''
    from tensorflow.keras.applications.resnet50  import ResNet50
    from tensorflow.keras.models import load_model, Model
    from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
      
    base_model = ResNet50(input_shape=(128,128,3),
                           include_top=False,
                           # weights="imagenet",
                           weights='resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
#                            alpha=1.0,
#                            dropout=1e-3)
                         )
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(64)(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    optimizer = keras.optimizers.Adam(lr=learning_rate)
    
    # 不能使用Accuracy作为衡量指标                   # keras.metrics.mean_squared_error
    model.compile(optimizer=optimizer, metrics=[keras.metrics.mean_squared_error], loss=keras.losses.mean_squared_error)
    
    return model


def _parse(example):
    """从tfrecord文件中提取训练数据
    Args:
        example: protobuf文件.

    Returns:
        data-label对.
    """
    keys_to_features = {
        'image/filename': tf.io.FixedLenFeature([], tf.string),
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'label/marks': tf.io.FixedLenFeature([64], tf.float32),
    }
    parsed_features = tf.io.parse_single_example(example, keys_to_features)

    # Extract features from single example
    image_decoded = tf.image.decode_image(parsed_features['image/encoded'])
    image_reshaped = tf.reshape(
        image_decoded, [IMG_HEIGHT, IMG_WIDTH, IMG_CHANNEL])
    image_float = tf.cast(image_reshaped, tf.float32)
    points = tf.cast(parsed_features['label/marks'], tf.float32)

    return image_float, points


def get_parsed_dataset(record_file, batch_size, epochs=None, shuffle=True):
    """返回tf.dataset以供训练
    Args:
        record_file: TFRecord file.
        batch_size: batch size.
        epochs: epochs of dataset.
        shuffle: whether to shuffle the data.

    Returns:
        Pparsed tf.dataset.
    """
    dataset = tf.data.TFRecordDataset(record_file)

    if shuffle is True:
        dataset = dataset.shuffle(buffer_size=10000)
    dataset = dataset.map(
        _parse, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.repeat(epochs)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset


def run():
    """训练与储存模型"""

    # Create the Model
    mark_model = get_compiled_model(MARK_SIZE*2)

    callbacks = [keras.callbacks.TensorBoard(log_dir='./log', update_freq=1024),
#                  BestLogger('/home/tione/log/best.json'),
                 keras.callbacks.ModelCheckpoint(filepath='./train',
                                                 monitor='loss',
                                                 save_freq='epoch')]
    if True:
        train_dataset = get_parsed_dataset(record_file='./tfrecord/train.record',
                                           batch_size=BATCH_SIZE,
                                           epochs=EPOCHS,
                                           shuffle=True)
        print('Starting to train.')
        _ = mark_model.fit(train_dataset,
                           epochs=EPOCHS,
#                            steps_per_epoch=TRAINING_STEPS,
                           callbacks=callbacks)

    if True:
        print('Starting to evaluate.')
        eval_dataset = get_parsed_dataset(record_file="./tfrecord/validation.record",
                                          batch_size=BATCH_SIZE,
                                          epochs=1,
                                          shuffle=False)
        evaluation = mark_model.evaluate(eval_dataset)
        print(evaluation)

    if True:
        #print("Saving model to directory: {}".format("/home/tione/saved_model1"))
        mark_model.save("saved_resnet.h5")
        #mark_model.save_weights('/home/tione/saved_model/easy_checkpoint')

In [117]:
!pip install pydot
!pip install pydotplus
!apt-get install graphviz
!pip install graphviz
handout = get_compiled_model(0.001,MARK_SIZE*2)
keras.utils.plot_model(handout, "resnet50.png")

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


##### 2. 定义优化函数（选择回归模型，定义黑盒函数, 定义acquisition函数）    

In [135]:
# 黑盒函数
def blackbox(train_dataset,tuned_learning_rate):
    """
    Attribute: Resource-expensive
    
    Arguments:
    x -- image data.
    y -- labels.
    learning_rate -- learning rate为超参数

    Returns:
    accuracy of the trained model under selected learning rate.
    """
    epochs = 1
    model = get_compiled_model(MARK_SIZE*2, tuned_learning_rate)

    # 训练在tuned_learning_rate下的模型
    trained_model_history = model.fit(train_dataset,
                         epochs=epochs,
                         verbose=0
                         )
    
    loss = trained_model_history.history['loss'][-1]
    print(loss)
    
    # 删除模型
    del model

    # 删除session，详见：https://www.coder.work/article/97730
    K.clear_session()

    return loss

In [101]:
# 返回在X处的expected improvement
def expected_improvement(X, X_sample, Y_sample, gpr, xi=0.0001): # xi=0.01
    '''
    Computes the EI at points X based on existing samples X_sample
    and Y_sample using a Gaussian process surrogate model. 
    注: EI is Expected Improvement; GP is Gaussian process.

    Arguments:
        X -- Points at which EI shall be computed.
        X_sample -- Sample locations.
        Y_sample -- Sample values.
        gpr -- A GaussianProcessRegressor fitted to samples.
        xi -- Exploitation-exploration trade-off parameter.

    Returns:
        Expected improvements at points X.
    '''

    # 基于上一轮的高斯过程拟合，计算所有点的均值和标准差
    mu, sigma = gpr.predict(X, return_std=True)

    sigma = sigma.reshape(-1, 1)

    # 计算最低loss
    sample_opt = np.min(Y_sample)

    # 用errstate防止计算问题而崩溃
    with np.errstate(divide='warn'):
        imp = mu - sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0

    return ei

In [102]:
# 基于现有猜测集合，预计下一个最佳X值的坐标
def sample_next_hyperparameter(acquisition, X_sample, Y_sample, gpr, bounds, n_restarts=50):
    '''
    Proposes the next hyperparameter point by optimizing
    the acquisition function.

    Args:
        acquisition -- Acquisition function.
        X_sample -- Sample locations.
        Y_sample -- Sample values.
        gpr -- A GaussianProcessRegressor fitted to samples.
        bounds -- Bounds of the hyperparameter.
        n_restarts -- Number of restarts for finding the optimum.

    Returns:
        Location of the acquisition function maximum.
    '''

    dim = X_sample.shape[1]
    min_val = float('inf')
    min_x = None

    def min_obj(X):
        # Minimization objective is the negative acquisition function
        return -acquisition(X.reshape(-1, dim), X_sample, Y_sample, gpr)

    # Find the best optimum by starting from n_restart different random points.
    for x0 in np.random.uniform(
            bounds[:, 0], bounds[:, 1], size=(n_restarts, dim)):
        res = minimize(min_obj, x0=x0, bounds=bounds, method='L-BFGS-B')
        if res.fun < min_val:
            min_val = res.fun[0]
            min_x = res.x

    return min_x.reshape(-1, 1)

3. 获取Dataset，获取初始X值

In [136]:
# 获取dataset
train_dataset = get_parsed_dataset(record_file='./tfrecord/train.record',
                                           batch_size=BATCH_SIZE,
                                           epochs=EPOCHS,
                                           shuffle=True)
eval_dataset = get_parsed_dataset(record_file="./tfrecord/validation.record",
                                          batch_size=BATCH_SIZE,
                                          epochs=1,
                                          shuffle=False)
print(train_dataset)
print(eval_dataset)

<DatasetV1Adapter shapes: ((?, 128, 128, 3), (?, 64)), types: (tf.float32, tf.float32)>
<DatasetV1Adapter shapes: ((?, 128, 128, 3), (?, 64)), types: (tf.float32, tf.float32)>


In [137]:
# 获取初始X值与对应的Y值

# bounds为学习速率可能范围
bounds = np.array([[1e-5, 1]])

#根据经验设置初始学习速率 获取对应Y值
X_init = np.array([[1e-3], [2e-1]])
# Testing only: Y_init = np.array([[98e-4],[77e-4]])
Y_init = np.vstack((blackbox(train_dataset, X_init[0][0]),
                    blackbox(train_dataset, X_init[1][0])))
Y_init = np.array(Y_init).reshape(-1, 1)

ValueError: The `batch_size` argument must not be specified for the given input type. Received input: <DatasetV1Adapter shapes: ((?, 128, 128, 3), (?, 64)), types: (tf.float32, tf.float32)>, batch_size: 32

4. 迭代并获取最佳X值

In [12]:
gpr_kernel = ConstantKernel(1.0) * Matern(length_scale=1, nu=2.5)
gpr = GaussianProcessRegressor(kernel=gpr_kernel)
# Initialize samples
X_sample = X_init
Y_sample = Y_init

# Number of iterations
n_iter = 10

for i in range(n_iter):
    # Update Gaussian process with existing samples
    gpr.fit(X_sample, Y_sample)

    # Obtain next sampling point from the acquisition function
    # (expected_improvement)
    X_next = sample_next_hyperparameter(
        expected_improvement, X_sample, Y_sample, gpr, bounds)

    # Obtain next noisy sample from the objective function
    Y_next = blackbox(train_dataset, X_next[0][0])

    # Add sample to previous samples
    X_sample = np.vstack((X_sample, X_next))
    Y_sample = np.vstack((Y_sample, Y_next))

NameError: name 'Y_init' is not defined

5. 结果

In [ ]:
print("Sampled learning rates:\n", np.hstack((X_sample, Y_sample)))
best_learning_rate = X_sample[Y_sample.argmax()][0]
print("Best learning rate: ", best_learning_rate)

###### source
###### 1. https://github.com/ohquai/ResNet_cifar10_TensorFlow 对ResNet优化的启发
###### 2. 参考了 https://github.com/thuijskens/bayesian-optimization 中高斯过程的代码
###### 3. 重点参考了下文：https://static.sigopt.com/773979031a2d61595b9bda23bb81a192341f11a4/pdf/SigOpt_Bayesian_Optimization_Primer.pdf

#### 候选方案：贝叶斯优化库实现超参数优化：

In [118]:
# 黑盒函数
def blackbox(learning_rate):
    epochs = 3
    model = get_compiled_model(MARK_SIZE*2, learning_rate)

    blackbox = model.fit(train_dataset,epochs=epochs)
    loss = blackbox.history['loss'][-1]

    # delete the Keras model with these hyper-parameters from memory.
    del model

    # clear the keras session
    K.clear_session()

    return -loss

In [126]:
# 使用封装好的库
! pip install bayesian-optimization

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
Looking in indexes: http://mirrors.tencentyun.com/pypi/simple


In [127]:
from bayes_opt import BayesianOptimization

SyntaxError: invalid syntax (domain_reduction.py, line 11)

In [123]:
pbounds = {'learning_rate': (1e-5, 1)}
optimizer = BayesianOptimization(f=blackbox,#导入黑盒算法
pbounds=pbounds,random_state=1)
optimizer.maximize(init_points=2,n_iter=10)
print(optimizer.max)#查看最优参数及表现
for i, res in enumerate(optimizer.res):#查看其他参数及表现
    print("Iteration {}: \n\t{}".format(i, res))

NameError: name 'BayesianOptimization' is not defined

##### source: https://github.com/fmfn/BayesianOptimization